In [1]:
from routingcomparison.routingapp.common.network_stat_utils import get_host
get_host()

ModuleNotFoundError: No module named 'routingcomparison'

In [11]:
def get_host(max_display_mac=-1):
    # I have to some dirty hack to remove invalid hosts
    hosts = rq.get('http://0.0.0.0:8080/hosts').json()
    if max_display_mac > 0: 
        hosts = {'hosts': [host for host in hosts['hosts'] if mac_to_int(host['mac']) < 100]}
    return hosts
get_host()

{'hosts': [{'mac': '00:00:00:00:00:04',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:4'],
   'port': {'dpid': '0000000000000004',
    'port_no': '00000001',
    'hw_addr': '3a:98:26:30:31:28',
    'name': 's4-eth1'}},
  {'mac': '00:00:00:00:0f:a0',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:fa0'],
   'port': {'dpid': '0000000000000004',
    'port_no': '00000002',
    'hw_addr': '26:fa:b7:44:f4:75',
    'name': 's4-eth2'}},
  {'mac': '00:00:00:00:17:70',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:1770'],
   'port': {'dpid': '0000000000000006',
    'port_no': '00000002',
    'hw_addr': '5e:49:e9:30:59:5f',
    'name': 's6-eth2'}},
  {'mac': '00:00:00:00:00:05',
   'ipv4': [],
   'ipv6': ['::', 'fe80::200:ff:fe00:5'],
   'port': {'dpid': '0000000000000005',
    'port_no': '00000001',
    'hw_addr': '42:b5:d5:92:b8:02',
    'name': 's5-eth1'}},
  {'mac': '00:00:00:00:07:d0',
   'ipv4': ['10.0.0.8'],
   'ipv6': ['::', 'fe80::200:ff:fe00:7d0'],
   'port': {'dpid': '0

In [12]:
import requests
import time
import json
import requests as rq
import random
from common.traffic_gen_utils import PingTraffic, HttpFileTransferController

# Generate list of host
response = rq.get('http://0.0.0.0:8000/host', [])

# Check for successful response
if response.status_code == 200:
  # Parse the response data (assuming it's JSON)
  hosts_data = response.json()
else:
  print("Error retrieving hosts:", response.status_code)
  hosts_data = []  # Set to empty list if there's an error


routes = []
hosts = []
for host_name, host_info in hosts_data.items():
    if host_name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
        # Add host to list of hosts but in integer
        hosts.append(int(host_name[1:]))
# print(hosts)

for _ in range(0, 2):
  for _ in range(0, 10):
      src_host = random.choice(hosts)
      dst_host = random.choice(hosts)
      while src_host == dst_host:
          dst_host = random.choice(hosts)
      routes.append({'src_host': src_host, 'dst_host': dst_host})
  
  json.dumps({"route": routes})

  sec_morl = rq.post('http://0.0.0.0:8001/single/rl/sec',
          data=json.dumps({"route": routes})).json()
  print(sec_morl)
  
  for route in routes:
    result_ping = PingTraffic().ping_single(f'h{route["src_host"]}', f'h{route["dst_host"]}').json()
    
    time.sleep(1)
    

# min_hop = rq.post('http://0.0.0.0:8001/single/min_hop',
#         data=json.dumps({"route": routes})).json()
# print(min_hop)
# PingTraffic().ping_single('h1', 'h2')


[{'status': 200, 'flowrule': {'dpid': 6, 'cookie': 1, 'cookie_mask': 1, 'table_id': 0, 'idle_timeout': 3000, 'hard_timeout': 3000, 'priority': 1, 'flags': 1, 'match': {'in_port': 1, 'dl_src': '00:00:00:00:00:06', 'dl_dst': '00:00:00:00:00:04'}, 'actions': [{'type': 'OUTPUT', 'port': 3}]}}, {'status': 200, 'flowrule': {'dpid': 6, 'cookie': 1, 'cookie_mask': 1, 'table_id': 0, 'idle_timeout': 3000, 'hard_timeout': 3000, 'priority': 1, 'flags': 1, 'match': {'in_port': 3, 'dl_src': '00:00:00:00:00:04', 'dl_dst': '00:00:00:00:00:06'}, 'actions': [{'type': 'OUTPUT', 'port': 1}]}}, {'status': 200, 'flowrule': {'dpid': 2, 'cookie': 1, 'cookie_mask': 1, 'table_id': 0, 'idle_timeout': 3000, 'hard_timeout': 3000, 'priority': 1, 'flags': 1, 'match': {'in_port': 4, 'dl_src': '00:00:00:00:00:06', 'dl_dst': '00:00:00:00:00:04'}, 'actions': [{'type': 'OUTPUT', 'port': 3}]}}, {'status': 200, 'flowrule': {'dpid': 2, 'cookie': 1, 'cookie_mask': 1, 'table_id': 0, 'idle_timeout': 3000, 'hard_timeout': 3000,